In [22]:
import cv2
import face_recognition
import pickle
import os
import numpy as np
import time
known_face_encodings = []
known_face_names = []
try:
    with open('known_faces.pkl', 'rb') as f:
        known_face_encodings, known_face_names = pickle.load(f)
except (FileNotFoundError, EOFError):
    train_dir = "C:\\Users\\pranamya\\Desktop\\dataset2"
    if not os.path.exists(train_dir):
        print(f"Training directory '{train_dir}' does not exist.")
        exit()
    for person_dir in os.listdir(train_dir):
        person_name = person_dir
        person_images = os.listdir(os.path.join(train_dir, person_dir))
        for person_image in person_images:
            image = face_recognition.load_image_file(os.path.join(train_dir, person_dir, person_image))
            face_encoding = face_recognition.face_encodings(image)
            if len(face_encoding) == 0:
                print(f"No face found in {person_image}")
                continue
            face_encoding = face_encoding[0]
            known_face_encodings.append(face_encoding)
            known_face_names.append(person_name)
    with open('known_faces.pkl', 'wb') as f:
        pickle.dump((known_face_encodings, known_face_names), f)

In [23]:
# Using real time video capturing
# Start the video capture from the webcam
person_start_times = {}
video_capture = cv2.VideoCapture(0)
present_students = set()
while True:
    # Capture frame-by-frame
    ret, frame = video_capture.read()
    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # Detect faces in the frame
    faces = face_recognition.face_locations(gray)
    # Iterate over the detected faces
    for (top, right, bottom, left) in faces:
        # Draw a rectangle around the face
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
        # Extract the face encoding from the current frame
        face_encoding = face_recognition.face_encodings(frame, [(top, right, bottom, left)])[0]
        # Compare the face encoding with the encodings of known faces
        face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
        min_distance_index = np.argmin(face_distances)
        if face_distances[min_distance_index] < 0.5:
            name = known_face_names[min_distance_index]
            if name not in person_start_times:
                person_start_times[name] = time.time()
            present_students.add(name)
        else:
            name = "Unknown"
        # Draw the name of the person below the face rectangle
        cv2.putText(frame, name, (left, bottom + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 1)
    # Display the resulting frame
    cv2.imshow('Video', frame)
    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
print("Present Students:")
for student in present_students:
    local_time = time.localtime(person_start_times[student])
    print(student,' ',time.strftime('%d-%m-%Y %H:%M:%S', local_time))
# Release the video capture and destroy the windows
video_capture.release()
cv2.destroyAllWindows()

Present Students:
Ratikesh   26-08-2023 12:21:11


In [8]:
# Using image as an input

image = cv2.imread("C:\\Users\\pranamya\\Desktop\\dataset2\\Naira\\79b901d3-8017-4ca1-8d42-bff03fffa70a.jpg")
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
faces = face_recognition.face_locations(gray)
present_students = []

for (top, right, bottom, left) in faces:
    # Draw a rectangle around the face
    cv2.rectangle(image, (left, top), (right, bottom), (0, 255, 0), 2)

    # Extract the face encoding from the current frame
    face_encoding = face_recognition.face_encodings(image, [(top, right, bottom, left)])[0]

    # Compare the face encoding with the encodings of known faces
    face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
    min_distance_index = np.argmin(face_distances)
    if face_distances[min_distance_index] < 0.5:
        name = known_face_names[min_distance_index]
        present_students.append(name)
    else:
        name = "Unknown"

    # Draw the name of the person below the face rectangle
    cv2.putText(image, name, (left, bottom + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 1)

print("Present Students:")
for student in present_students:
    print(student)

cv2.imshow('Image', image)
cv2.waitKey(0) & 0xFF == ord('q')     
cv2.destroyAllWindows()


Present Students:
Naira
